<a href="https://colab.research.google.com/github/g-lam/colab-samples/blob/main/Text-to-Text/Llama_2_GGUF_CTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!lscpu

In [ ]:
!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers

In [ ]:
from ctransformers import AutoModelForCausalLM

config = {'max_new_tokens': 256, 'repetition_penalty': 1.1, 'temperature': 0.1, 'stream': True}

models = [
    {
        'name': "TheBloke/Llama-2-7b-Chat-GGUF",
        'file': "llama-2-7b-chat.Q4_K_M.gguf",
        'layers': 110 #110 for 7b, 130 for 13b
    },
    {
        'name': "TheBloke/Llama-2-13B-chat-GGUF",
        'file': "llama-2-13b-chat.Q4_K_M.gguf",
        'layers': 130 #110 for 7b, 130 for 13b
    },
    {
        'name': "TheBloke/Llama-2-70B-chat-GGUF",
        'file': "llama-2-70b-chat.Q5_K_S.gguf",
        'layers': 150 #110 for 7b, 130 for 13b
    },
]

In [ ]:
model_index = 0

llm = AutoModelForCausalLM.from_pretrained(models[model_index]['name'],
                                           model_type="llama",
                                           model_file=models[model_index]['file'],
                                           #lib='avx2', #for cpu use
                                           gpu_layers=models[model_index]['layers'],
                                           **config
                                           )


## LLAMA-2 Model Tests

### Tokenizer

In [ ]:
prompt="""Write a poem to help me remember the first 10 elements on the periodic table, giving each
element its own line."""

In [ ]:
tokens = llm.tokenize(prompt)

In [ ]:
tokens

### Pipeline

In [ ]:
# 'pipeline' execution
llm(prompt, stream=False)

In [ ]:
prompt2 = """Quando e por quem o Brasil foi descoberto?"""
llm(prompt2, stream=False)

### Generate with stream execution

In [ ]:
# LlAMA-2-7b-chat execution
import time
start = time.time()
NUM_TOKENS=0
print('-'*4+'Start Generation'+'-'*4)
for token in llm.generate(tokens):
    print(llm.detokenize(token), end='', flush=True)
    NUM_TOKENS+=1
time_generate = time.time() - start
print('\n')
print('-'*4+'End Generation'+'-'*4)
print(f'Num of generated tokens: {NUM_TOKENS}')
print(f'Time for complete generation: {time_generate}s')
print(f'Tokens per secound: {NUM_TOKENS/time_generate}')
print(f'Time per token: {(time_generate/NUM_TOKENS)*1000}ms')

## LangChain Integration

https://python.langchain.com/docs/ecosystem/integrations/ctransformers